In [1]:
from typing import Dict, List
from ddnn.nn import *
from ddnn.data import *
from ddnn.validation import *

In [2]:
class Logger:
    def __init__(self, estimator: Estimator, losses: List[str]):
        self._estimator = estimator
        self._losses = losses
        self._scores = []

    def update_hp(self, hp_dict : Dict):
        self._scores.append({"hp": hp_dict.copy(), "folds": []})

    def update_fold(self, fold_dict : Dict):
        self._vfold = fold_dict["test"]
        self._tfold = fold_dict["train"]

        current_hp_config = self._scores[-1]
        current_hp_config["folds"].append({"train":{},"valid":{}})

        current_fold = current_hp_config["folds"]
        print(type(current_fold["train"]))
        for loss in self._losses:
            current_fold["train"][loss] = []
            current_fold["valid"][loss] = []

    def __call__(self, dict):
        dt = self._estimator.evaluate(self._losses, self._tfold)
        dv = self._estimator.evaluate(self._losses, self._tfold)

        current_train = self._scores[-1]["folds"]["train"]
        current_valid = self._scores[-1]["folds"]["valid"]
        
        for loss in self._loss_names:
            current_train[loss].append(dt[loss])
            current_valid[loss].append(dt[loss])
        
        

In [3]:
net = NeuralNetwork(
    [
        LinearLayer((8, 16)),
        ActivationFunction(),
        LinearLayer((16, 16)),
        ActivationFunction(),
        LinearLayer((16, 2)),
    ]
)
estimator = Estimator(net) # fix to work without anything
grid = {}
grid["layers"] = [[(4, "ReLU"), (1, "logistic")]]
grid["learning_rate"] = [0.5, 0.6]
grid["momentum_coefficient"] = [0.9]
grid["optimizer"] = ["SGD"]
grid["loss"] = ["MSE"]
grid["l2_coefficient"] = [1e-3, 1e-4, 1e-5]
grid["batchsize"] = [-1, 16]
grid["weight_initializer"] = ["random_uniform"]
# grid['fan_mode'] = ['fan_in', 'fan_out']
selector = GridSearch(estimator, grid, 123)

In [4]:
# data = read_ML_cup("train")
data = read_monks(1, "train")
data = onehot_encoding(data=data)

In [5]:
logger = Logger(estimator, ["MSE", "binary_accuracy"])

In [6]:
selector.k_fold(
    data,
    5,
    n_epochs=2000,
    loss_list=["MSE"],
    early_stopping=(10, 10),
    seed=123,
    on_fold_change=logger.update_fold,
    on_hp_change=logger.update_hp,
    training_callback=logger,
)

{'layers': [(4, 'ReLU'), (1, 'logistic')], 'learning_rate': 0.5, 'momentum_coefficient': 0.9, 'optimizer': 'SGD', 'loss': 'MSE', 'l2_coefficient': 0.001, 'batchsize': -1, 'weight_initializer': 'random_uniform'}


TypeError: list indices must be integers or slices, not str